In [1]:
import numpy as np
# from utils import *
import random
import pprint
import copy

In [2]:
data=open('girls.txt','r').read()
print(data)
data=data.lower()
chars=list(set(data))
data_size,vocab_size = len(data),len(chars)

print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

Olivia
Amelia
Isla
Ava
Lily
Ivy
Freya
Florence
Isabella
Mia
Willow
Sienna
Poppy
Sophia
Elsie
Rosie
Grace
Millie
Emily
Sofia
Daisy
Evelyn
Evie
Phoebe
Ella
Charlotte
Harper
Maya
Matilda
Ruby
Aria
Penelope
Hallie
Luna
Bonnie
Ada
Emilia
Alice
Sophie
Esme
Isabelle
Maisie
Violet
Delilah
Mila
Eva
Arabella
Maeve
Aurora
Mabel
Ayla
Lottie
Eliza
Rose
Erin
Layla
Olivia
Amelia
Isla
Ava
Lily
Ivy
Freya
Florence
Isabella
Sienna
Mia
Willow
Poppy
Sophia
Elsie
Grace
Rosie
Emily
Millie
Sofia
Daisy
Evelyn
Evie
Phoebe
Charlotte
Ella
Harper
Maya
Matilda
Ruby
Aria
Penelope
Luna
Hallie
Ada
Alice
Bonnie
Emilia
Sophie
Esme
Isabelle
Maisie
Delilah
Violet
Eva
Mila
Arabella
Maeve
Aurora
Mabel
Ayla
Eliza
Rose
Lottie
Margot
Chloe
Layla
Scarlett
Elizabeth
Eleanor
Erin
Orla
Imogen
Iris
Harriet
Emma
Thea
Nancy
Jessica
Zara
Robyn
Maryam
Olive
Maria
Elodie
Lola
Bella
Lyra
Gracie
xlana
kFatima
tuqa
Lyla
Ophelia
Ottilie
Ellie
Molly
Lara
Myla
Eden
Darcie
Eloise
Nova
Clara
Lucy
Lilly
Jasmine
Amelie
Edith
Summer
Fatima
Hannah


In [3]:
chars = sorted(chars)
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

{   0: '\n',
    1: 'a',
    2: 'b',
    3: 'c',
    4: 'd',
    5: 'e',
    6: 'f',
    7: 'g',
    8: 'h',
    9: 'i',
    10: 'j',
    11: 'k',
    12: 'l',
    13: 'm',
    14: 'n',
    15: 'o',
    16: 'p',
    17: 'q',
    18: 'r',
    19: 's',
    20: 't',
    21: 'u',
    22: 'v',
    23: 'w',
    24: 'x',
    25: 'y',
    26: 'z'}


In [4]:
def clip(gradients, maxValue):
    gradients = copy.deepcopy(gradients)
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient,-maxValue,maxValue,out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [5]:
def sample(parameters, char_to_ix, seed):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    x = np.zeros([vocab_size, 1])

    a_prev = np.zeros([n_a, 1])

    indices = []

    idx = -1

    counter = 0
    newline_character = char_to_ix['\n']

    while (idx != newline_character and counter != 50):

        a = np.tanh(np.dot(Wax, x)+np.dot(Waa, a_prev)+b)
        z = np.dot(Wya, a)+by
        y = softmax(z)

        np.random.seed(counter + seed)

        idx = np.random.choice(range(len(y)), p=y.ravel())

        indices.append(idx)

        x = np.zeros((vocab_size, 1))
        x[idx] = 1

        a_prev = a

        seed += 1
        counter += 1

    if (counter == 50):
        indices.append(char_to_ix['\n'])

    return indices

In [6]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def rnn_forward(X, Y, a0, parameters, vocab_size=27):

    x, a, y_hat = {}, {}, {}

    a[-1] = np.copy(a0)

    loss = 0

    for t in range(len(X)):

        x[t] = np.zeros((vocab_size, 1))
        if (X[t] != None):
            x[t][X[t]] = 1

        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])

        loss -= np.log(y_hat[t][Y[t], 0])

    cache = (y_hat, a, x)

    return loss, cache


def rnn_backward(X, Y, parameters, cache):

    gradients = {}

    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']

    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(
        Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])

    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(
            dy, gradients, parameters, x[t], a[t], a[t-1])

    return gradients, a

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b'] += -lr * gradients['db']
    parameters['by'] += -lr * gradients['dby']
    return parameters


def initialize_parameters(n_a, n_x, n_y):
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01  # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01  # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01  # hidden to output
    b = np.zeros((n_a, 1))  # hidden bias
    by = np.zeros((n_y, 1))  # output bias

    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

    return parameters


def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length


def get_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character
    return txt

In [7]:
def optimize(X, Y, a_prev, parameters, learning_rate=0.01):

    loss, cache = rnn_forward(X, Y, a_prev, parameters)

    gradients, a = rnn_backward(X, Y, parameters, cache)

    gradients = clip(gradients, 5)

    parameters = update_parameters(parameters, gradients, learning_rate)

    return loss, gradients, a[len(X)-1]

In [8]:
def model(data_x, ix_to_char, char_to_ix, num_iterations=35000, n_a=50, dino_names=7, vocab_size=27, verbose=False):

    n_x, n_y = vocab_size, vocab_size

    parameters = initialize_parameters(n_a, n_x, n_y)

    loss = get_initial_loss(vocab_size, dino_names)

    examples = [x.strip() for x in data_x]

    np.random.seed(0)
    np.random.shuffle(examples)

    a_prev = np.zeros((n_a, 1))

    last_dino_name = "abc"

    for j in range(num_iterations):

        idx = j % len(examples)

        single_example = examples[idx]
        single_example_chars = [c for c in single_example]
        single_example_ix = [char_to_ix[c] for c in single_example]
        X = [None] + single_example_ix

        ix_newline = char_to_ix["\n"]
        Y = X[1:] + [ix_newline]

        curr_loss, gradients, a_prev = optimize(
            X, Y, a_prev, parameters, learning_rate=0.01)

        if verbose and j in [0, len(examples) - 1, len(examples)]:
            print("j = ", j, "idx = ", idx,)
        if verbose and j in [0]:
            print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")

        loss = loss*0.999 + curr_loss*0.001

        if j % 2000 == 0:

            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')

            seed = 0
            for name in range(dino_names):

                sampled_indices = sample(parameters, char_to_ix, seed)
                last_dino_name = get_sample(sampled_indices, ix_to_char)
                print(last_dino_name.replace('\n', ''))

                seed += 1

            print('\n')

    return parameters, last_dino_name

In [9]:
parameters, last_name = model(data.split("\n"), ix_to_char, char_to_ix, 22001, verbose = True)

j =  0 idx =  0
single_example = emily
single_example_chars ['e', 'm', 'i', 'l', 'y']
single_example_ix [5, 13, 9, 12, 25]
 X =  [None, 5, 13, 9, 12, 25] 
 Y =        [5, 13, 9, 12, 25, 0] 

Iteration: 0, Loss: 23.067562

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


j =  203 idx =  203
j =  204 idx =  0
Iteration: 2000, Loss: 16.798945

Liyrsobiea
Hla
Iyrsobie
La
Ysrobie
Ca
Ttpa


Iteration: 4000, Loss: 13.971144

Leysoramain
Hie
Hytvia
La
Vovia
Eba
Spoe


Iteration: 6000, Loss: 12.022943

Liytnelie
Ilie
Ivyemalie
La
Vynia
Eka
Runa


Iteration: 8000, Loss: 10.433617

Litwesalie
Hia
Ivyneelemmayen
Lie
Tyne
Da
Rpoe


Iteration: 10000, Loss: 9.145548

Ilyve
Ella
Eusloe
Ila
Syroetemelya
A
Osna


Iteration: 12000, Loss: 8.482785

Lotttielinabrise
Elie
Ettie
Lhae
Syryam
A
Ruby


Iteration: 14000, Loss: 8.146829

Elle
Elie
Eusy
Ela
Syote
Aba
Pyla


Iteration: 16000, Loss: 7.499963

Elyn
Elia
Eryne
Elia
Syra
Chaelot
Ottil


Iterati

In [10]:
# print(parameters)

In [11]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from uvicorn import Config, Server
import asyncio
app = FastAPI()


class ModelInput(BaseModel):
    seed: int
    num_names: int = 5


config = Config(app)
server = Server(config=config)
loop = asyncio.get_event_loop()
loop.create_task(server.serve())


@app.post("/generate_names/")
async def generate_names(input_data: ModelInput):
    global parameters, a_prev  # Use global if parameters are set during server startup
    if parameters is None:
        raise HTTPException(status_code=500, detail="Model is not initialized")

    generated_names = []
    seed = input_data.seed

    for _ in range(input_data.num_names):
        sampled_indices = sample(parameters, char_to_ix, seed)
        generated_name = get_sample(sampled_indices, ix_to_char)
        generated_names.append(generated_name.replace('\n', ''))
        seed += 1

    return {"generated_names": generated_names}

In [14]:
# from fastapi import FastAPI, WebSocket, HTTPException
# from pydantic import BaseModel
# import numpy as np

# # Initialize FastAPI
# app = FastAPI()

# def generate_name_from_input(input_chars, seed=0):
#     global parameters
#     if parameters is None:
#         raise ValueError("Model parameters are not initialized.")
    
#     # Example generation logic based on input characters
#     sampled_indices = sample(parameters, char_to_ix, seed)
#     generated_name = get_sample(sampled_indices, ix_to_char)
#     return generated_name.replace('\n', '')

# # WebSocket endpoint for real-time communication
# @app.websocket("/ws/generate_name")
# async def websocket_endpoint(websocket: WebSocket):
#     await websocket.accept()
#     try:
#         while True:
#             data = await websocket.receive_text()
#             if not data:
#                 break
#             generated_name = generate_name_from_input(data)
#             await websocket.send_text(generated_name)
#     except Exception as e:
#         await websocket.close()
#         raise HTTPException(status_code=500, detail=str(e))


INFO:     127.0.0.1:23313 - "GET /ws/generate_name HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:23314 - "POST /generate_names/ HTTP/1.1" 200 OK


INFO:     127.0.0.1:23340 - "GET /ws/generate_name HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:23341 - "POST /generate_names/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:23351 - "POST /generate_names/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:23354 - "POST /generate_names/ HTTP/1.1" 200 OK


In [ ]:
# pip install 'uvicorn[standard]'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'uvicorn[standard]'"
